# Section 6: No More Toy-Examples/Optimization Prize

Dataset of insurance charges in the US. Link to [dataset](https://www.kaggle.com/mirichoi0218/insurance)

## Set-up:

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import math

In [ ]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Data pre-processing:

In [ ]:
df = pd.read_csv('../../data/insurance.csv')
df.head(5)

## Regression:

In [ ]:
# function from section 2
def get_MAE(theta, X, y):
    ypred = X@theta.T
    mae = np.average(np.abs(ypred - y), axis=0)
    
    # compare to MAE from sklearn:
    mae_sklr = mean_absolute_error(X@theta.T, y)
    assert(math.isclose(mae, mae_sklr))
    return mae

In [ ]:
# Function to solve LPs from section 2 (mae penalized regression)
def solve_LP1(X, Y, lambda_):
    d = X.shape[1]
    N = X.shape[0]

    # auxiliary variables:
    Z = cp.Variable((N, 1))
    delta = cp.Variable((d, 1))

    # variable to solve:
    theta = cp.Variable((1, d))

    # linear program:
    prob = cp.Problem(cp.Minimize(cp.sum(Z) + lambda_ * cp.sum(delta)), [
        Y - X @ theta.T <= Z, -Y + X @ theta.T <= Z, theta <= delta,
        -theta <= delta
    ])

    # solve LP:
    prob.solve(solver=cp.GLPK)
    theta_opt = theta.value
    opt_value = prob.value
    dual_value = prob.constraints[0].dual_value
    return theta_opt, opt_value, dual_value 

In [ ]:
# Function to solve LPs from section 4 (adversarial training)
def solve_LP2(X, Y, lambda_, k):
    d = X.shape[1]
    N = X.shape[0]

    # auxiliary variables:
    Beta = cp.Variable((N, 1))
    b = cp.Variable((d, 1))
    alpha = cp.Variable((1,1))

    # variables to solve:
    theta = cp.Variable((1, d))

    # linear program:
    prob = cp.Problem( cp.Minimize(k * alpha + cp.sum(Beta) + lambda_ * cp.sum(b)), 
                      [
        alpha + Beta >= 1/k * (Y - X @ theta.T),
        alpha + Beta >= -1/k * (Y - X @ theta.T),
        Beta >= 0,
        -b <= theta,
        theta <= b
    ])

    # solve LP:
    prob.solve(solver=cp.GLPK)
    theta_opt = theta.value
    alpha_opt = b.value
    
    opt_value = prob.value
    dual_value = prob.constraints[0].dual_value
    return theta_opt,  opt_value, dual_value 

### Prepare data:

In [ ]:
df = pd.read_csv('../../data/insurance.csv')

# make text columns into numerical vaulues:
sex = {'female':1, 'male':0}
regions = {'northeast':0, 'northwest':1, 'southeast':2, 'southwest':3}
smoker = {'yes':1, 'no':0}
df['sex'] = df['sex'].apply(lambda x: sex[x]).astype("category")
df['region'] = df['region'].apply(lambda x: regions[x]).astype("category")
df['smoker']= df['smoker'].apply(lambda x: smoker[x]).astype("category")
df.head(4)

In [ ]:
## Split into train-test set:
X = pd.DataFrame(df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']])
y = df['charges']
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    np.expand_dims(y, 1),
                                                    test_size=0.5,
                                                    random_state=0)

# Add bias column to data:
X_train = np.concatenate((X_train, np.ones((X_train.shape[0], 1))), axis=1)
X_test = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis=1)

### Lasso regression with cross-validation: 

In [ ]:
# Hyperparameters:
lambda_ = np.logspace(-5, -1, 50, base=10)

In [ ]:
X = pd.DataFrame(df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']])
y = df['charges']

# 75% split:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    np.expand_dims(y, 1),
                                                    test_size=0.25,
                                                    random_state=0)
# Add bias column to data:
X_train = np.concatenate((X_train, np.ones((X_train.shape[0], 1))), axis=1)
X_test = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis=1)

e_train, e_val, thetas, opt_val = [], [], [], []

# Cross-validation for 50 values of lambda:
for l in lambda_:
    theta_opt, opt_value, dual_value = solve_LP1(X_train, Y_train, l)

    #k = math.floor(0.75 * X_train.shape[0])
    #theta_opt2, opt_value2, dual_value2 = solve_LP2(X_train, Y_train, l, k)

    thetas.append(theta_opt)
    opt_val.append(opt_value)

    # evaluate on training set:
    e_train.append(get_MAE(theta_opt, X_train, Y_train))
    # evaluate on validation set:
    e_val.append(get_MAE(theta_opt, X_test, Y_test))

# take hyperparameter with smallest validation error:
best_lambda = lambda_[np.argmin(e_val)]
best_theta = thetas[np.argmin(e_val)]
best_value = opt_val[np.argmin(e_val)]

print('---Optimal values--')
print(f'Optimal lambda: {best_lambda}')
print(f'Optimal value: {best_value}')
print(f'Optimal theta:\n {best_theta}')

In [ ]:
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {}'.format(get_MAE(best_theta, X_train, Y_train)))
print('\n')
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {}'.format(get_MAE(best_theta, X_test, Y_test)))

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(lambda_, e_val, label = 'validation error')

ax.set_xscale('log')
plt.title('Validation error for different hyperparameters')
ax.set_xlabel('Lambda')
ax.set_ylabel('Validation error')
plt.legend()

### Linear regression in 2D: 

In [ ]:
df = pd.read_csv('../../data/insurance.csv')

# make text columns categorical:
sex = {'female': 1, 'male': 0}
regions = {'northeast': 0, 'northwest': 1, 'southeast': 2, 'southwest': 3}
smoker = {'yes': 1, 'no': 0}

df['sex'] = df['sex'].astype("category")
df['region'] = df['region'].astype("category")
df['smoker'] = df['smoker'].astype("category")
df['children'] = df['children'].astype("category")
df.head(4)

In [ ]:
def cross_validation(X_train, X_test, y_train, y_test, kind = 'mae'):
    e_train, e_val, thetas, opt_val = [], [], [], []

    # Hyperparameters:
    lambda_ = np.logspace(-5, -1, 20, base = 10)

    # Cross-validation for 50 values of lambda:
    for l in lambda_:
        
        if kind == 'minmax':
            k = math.floor(0.75 * X_train.shape[0])
            theta_opt, opt_value, dual_value = solve_LP2(X_train, y_train, l, k)
        else:
            theta_opt, opt_value, dual_value = solve_LP1(X_train, y_train, l)
        thetas.append(theta_opt)
        opt_val.append(opt_value)

        # evaluate on training set:
        e_train.append(get_MAE(theta_opt, X_train, y_train))
        
        # evaluate on validation set:
        e_val.append(get_MAE(theta_opt, X_test, y_test))

    # take hyperparameter with smallest validation error:
    best_lambda = lambda_[np.argmin(e_val)]
    best_theta = thetas[np.argmin(e_val)]
    best_value = opt_val[np.argmin(e_val)]

    print('---Optimal values--')
    print(f'Optimal lambda: {best_lambda}')
    print(f'Optimal value: {best_value}')
    print(f'Optimal theta:\n {best_theta}')
    return best_lambda, best_theta, best_value, e_val

In [ ]:
## Split into train-test set:
X = pd.DataFrame(df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']])
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    np.expand_dims(
                                                        df['charges'], 1),
                                                    test_size=0.25,
                                                    random_state=0)
# Add bias column to data:
X_train = np.concatenate((X_train, np.ones((X_train.shape[0], 1))), axis=1)
X_test = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis=1)

# create dataframes for plots:
df_train = pd.DataFrame(
    data=X_train,
    columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'bias'])
df_train = pd.concat([df_train, pd.DataFrame(y_train)], axis=1)
df_train.rename(columns={0: "charges"}, inplace=True)
df_test = pd.DataFrame(
    data=X_test,
    columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'bias'])
df_test = pd.concat([df_test, pd.DataFrame(y_test)], axis=1)
df_test.rename(columns={0: "charges"}, inplace=True)

#### Predict charges from bmi:

In [ ]:
# Cross-validation:
#model 1:
print('Mae penalized regression:')
best_lambda, best_theta, best_value, e_val = cross_validation(
    df_train[['bmi', 'bias']].values, df_test[['bmi', 'bias']].values, y_train,
    y_test)
#model 2:
print('Adversarial model regression:')
best_lambda2, best_theta2, best_value2, e_val2 = cross_validation(
    df_train[['bmi', 'bias']].values,
    df_test[['bmi', 'bias']].values,
    y_train,
    y_test,
    kind='minmax')

In [ ]:
x_train = df_train[['bmi', 'bias']].values
x_test = df_test[['bmi', 'bias']].values

ypred1, ypred2 = [], []
for i in range(len(x_train)):
    ypred1.append(best_theta @ x_train[i, :])
    ypred2.append(best_theta2 @ x_train[i, :])

Plot predictions:

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(25, 8))
lambda_ = np.logspace(-5, -1, 20, base=10)

hues = ['sex', 'smoker', 'region', 'children']
for i in range(len(hues) + 1):
    if i < 4:
        axs[i].set_title(f'{hues[i]}')
        axs[i].plot(df_train['bmi'], ypred1, label='model 1', color='red')
        axs[i].plot(df_train['bmi'], ypred2, label='model 2', color='blue')
        sns.scatterplot(data=df_train,
                        x='bmi',
                        y='charges',
                        hue=hues[i],
                        ax=axs[i])
        axs[i].set_xlabel('bmi')
        axs[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    if i > 0:
        axs[i].set_ylabel('')
    else:
        axs[i].set_ylabel('insurance charges')

    if i == 4:
        axs[i].plot(lambda_, e_val, label='model 1', color='red')
        axs[i].plot(lambda_, e_val2, label='model 2', color='blue')
        pos = list(lambda_).index(best_lambda)
        val = e_val[pos]
        axs[i].plot(best_lambda, val, 'x')
        axs[i].set_xscale('log')
        axs[i].set_title('Validation error')
        axs[i].set_xlabel('Lambda')
        axs[i].set_ylabel('')
        axs[i].tick_params(axis="y", direction="in", pad=-90)
    axs[i].legend()

#### Predict charges from age:

In [ ]:
# Cross-validation:
print('Mae penalized regression:')
best_lambda, best_theta, best_value, e_val = cross_validation(
    df_train[['age', 'bias']].values, df_test[['age', 'bias']].values, y_train,
    y_test)

print('Adversarial model regression:')
best_lambda2, best_theta2, best_value2, e_val2 = cross_validation(
    df_train[['age', 'bias']].values,
    df_test[['age', 'bias']].values,
    y_train,
    y_test,
    kind='minmax')

In [ ]:
x_train = df_train[['age', 'bias']].values
x_test = df_test[['age', 'bias']].values

ypred1, ypred2 = [], []
for i in range(len(x_train)):
    ypred1.append(best_theta @ x_train[i, :])
    ypred2.append(best_theta2 @ x_train[i, :])

Plot results:

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(25, 8))
lambda_ = np.logspace(-5, -1, 20, base=10)

hues = ['sex', 'smoker', 'region', 'children']
for i in range(len(hues) + 1):
    if i < 4:
        axs[i].set_title(f'{hues[i]}')
        axs[i].plot(df_train['age'], ypred1, label='model 1', color='red')
        axs[i].plot(df_train['age'], ypred2, label='model 2', color='blue')
        sns.scatterplot(data=df_train,
                        x='age',
                        y='charges',
                        hue=hues[i],
                        ax=axs[i])
        axs[i].set_xlabel('age')
        axs[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    if i > 0:
        axs[i].set_ylabel('')
    else:
        axs[i].set_ylabel('insurance charges')

    if i == 4:
        axs[4].plot(lambda_, e_val, label='model 1', color='red')
        axs[4].plot(lambda_, e_val2, label='model 2', color='blue')
        pos = list(lambda_).index(best_lambda)
        val = e_val[pos]
        axs[4].plot(best_lambda, val, 'x')
        axs[4].set_xscale('log')
        axs[4].set_title('Validation error')
        axs[4].set_xlabel('Lambda')
        axs[4].set_ylabel('')
        axs[4].tick_params(axis="y", direction="in", pad=-50)

    axs[i].legend()

#### Regression with pre-processing:

##### Charges below \$10'000:

In [ ]:
X_train = df_train[df_train['charges'] <= 1e4]
X_test = df_test[df_test['charges'] <= 1e4]
y_train_ = np.expand_dims(X_train['charges'], 1)
y_test_ = np.expand_dims(X_test['charges'], 1)

# Cross-validation:
best_lambda, best_theta, best_value, e_val = cross_validation(
    X_train[['age', 'bias']].values, X_test[['age', 'bias']].values, y_train_,
    y_test_)

In [ ]:
x_train = X_train[['age', 'bias']].values
x_test = X_test[['age', 'bias']].values

ypred = []
for i in range(len(x_train)):
    ypred.append(best_theta @ x_train[i, :])

In [ ]:
fig, axs = plt.subplots(1,5, figsize = (25, 8))
lambda_ = np.logspace(-5, -1, 20, base = 10)

hues = ['sex', 'smoker', 'region', 'children']
for i in range(len(hues)+1): 
    if i < 4: 
        axs[i].set_title(f'{hues[i]}')
        axs[i].plot(X_train['age'], ypred, color='red')
        sns.scatterplot(data = X_train, x  = 'age', y= 'charges', hue = hues[i], ax = axs[i])
        axs[i].set_xlabel('age')
        axs[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0)) 
    
    if i >0:
        axs[i].set_ylabel('')
    else:
        axs[i].set_ylabel('insurance charges')
    
    if i == 4:
        axs[4].plot(lambda_, e_val)
        pos = list(lambda_).index(best_lambda)
        val = e_val[pos]
        axs[4].plot(best_lambda, val, 'x')
        axs[4].set_xscale('log')
        axs[4].set_title('Validation error')
        axs[4].set_xlabel('Lambda')
        axs[4].set_ylabel('')
        axs[4].tick_params(axis="y",direction="in", pad=-50)
        
    axs[i].legend()

##### Smokers and non-smokers:

In [ ]:
# Data for non smokers:
X_train_no = df_train[df_train['smoker'] == 'no']
X_test_no = df_test[df_test['smoker'] == 'no']
y_train_no = np.expand_dims(X_train_no['charges'], 1)
y_test_no = np.expand_dims(X_test_no['charges'], 1)

# Cross-validation:
best_lambda_no, best_theta_no, best_value_no, e_val_no = cross_validation(
    X_train_no[['bmi', 'bias']].values, X_test_no[['bmi', 'bias']].values,
    y_train_no, y_test_no)

In [ ]:
# Data for smokers:
X_train_yes = df_train[df_train['smoker'] == 'yes']
X_test_yes = df_test[df_test['smoker'] == 'yes']
y_train_yes = np.expand_dims(X_train_yes['charges'], 1)
y_test_yes = np.expand_dims(X_test_yes['charges'], 1)

# Cross-validation:
best_lambda_yes, best_theta_yes, best_value_yes, e_val_yes = cross_validation(
    X_train_yes[['bmi', 'bias']].values, X_test_yes[['bmi', 'bias']].values,
    y_train_yes, y_test_yes)

In [ ]:
x_train_yes = X_train_yes[['bmi', 'bias']].values
x_test_yes = X_test_yes[['bmi', 'bias']].values

x_train_no = X_train_no [['bmi', 'bias']].values
x_test_no = X_test_no [['bmi', 'bias']].values

ypred_yes, ypred_no = [], []
for i in range(len(x_train_yes)):
    ypred_yes.append(best_theta_yes @ x_train_yes[i, :])
    
for i in range(len(x_train_no)):
    ypred_no.append(best_theta_no @ x_train_no[i, :])

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (25, 8))
#plt.suptitle('Insurance charges depending on bmi')
lambda_ = np.logspace(-5, -1, 20, base = 10)

hues = ['smoker', 'smoker']
for i in range(3): 
    if i == 0: 
        axs[i].set_title(f'{hues[i]}')
        axs[i].plot(X_train_yes['bmi'], ypred_yes, color='red')
        sns.scatterplot(data = X_train_yes, x  = 'bmi', y= 'charges', hue = hues[i], ax = axs[i])
        axs[i].set_xlabel('bmi')
        axs[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0)) 
    if i == 1: 
        axs[i].set_title(f'Non {hues[i]}')
        axs[i].plot(X_train_no['bmi'], ypred_no, color='red')
        sns.scatterplot(data = X_train_no, x  = 'bmi', y= 'charges', hue = hues[i], ax = axs[i])
        axs[i].set_xlabel('bmi')
        axs[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 0)) 
    
    if i >0:
        axs[i].set_ylabel('')
    else:
        axs[i].set_ylabel('insurance charges')
    
    if i == 2:
        axs[i].plot(lambda_, e_val_yes, label = 'smokes')
        axs[i].plot(lambda_, e_val_no, label = 'doesn\'t smoke')
        
        pos = list(lambda_).index(best_lambda_yes)
        val_yes = e_val_yes[pos]
        axs[i].plot(best_lambda_yes, val_yes, 'x')
        
        pos = list(lambda_).index(best_lambda_no)
        val_no = e_val_no[pos]
        axs[i].plot(best_lambda_no, val_no, 'x')
        
        axs[i].set_xscale('log')
        axs[i].set_title('Validation error')
        axs[i].set_xlabel('Lambda')
        axs[i].set_ylabel('')
        axs[i].tick_params(axis="y",direction="in", pad=-50)
        
    axs[i].legend()
